In [8]:
#!pip install matplotlib-scalebar
#!pip install -U scikit-learn --upgrade

In [ ]:
from mpl_toolkits.mplot3d import Axes3D
import matplotlib.pyplot as plt
from matplotlib import cm
from matplotlib.ticker import LinearLocator, FormatStrFormatter
import numpy as np
from random import random, seed
from sklearn.model_selection import train_test_split
from matplotlib.ticker import MaxNLocator
import matplotlib.ticker as ticker
from mpl_toolkits.axes_grid1 import make_axes_locatable
from sklearn.utils import resample
from sklearn.model_selection import KFold
from sklearn.linear_model import Ridge
from sklearn.model_selection import cross_val_score
from sklearn.preprocessing import PolynomialFeatures
from imageio import imread
import seaborn as sns
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import Lasso
import warnings
warnings.filterwarnings("ignore")
from matplotlib_scalebar.scalebar import ScaleBar

In [ ]:
def FrankeFunction(x,y):
    term1 = 0.75*np.exp(-(0.25*(9*x-2)**2) - 0.25*((9*y-2)**2))
    term2 = 0.75*np.exp(-((9*x+1)**2)/49.0 - 0.1*(9*y+1))
    term3 = 0.5*np.exp(-(9*x-7)**2/4.0 - 0.25*((9*y-3)**2))
    term4 = -0.2*np.exp(-(9*x-4)**2 - (9*y-7)**2)
    return term1 + term2 + term3 + term4

def R2(y_data, y_model):
    return 1 - np.sum((y_data - y_model) ** 2) / np.sum((y_data - np.mean(y_data)) ** 2)

def MSE(y_data,y_model):
    n = np.size(y_model)
    return np.sum((y_data-y_model)**2)/n

In [ ]:
def design_matrix(x, y, n ):
    if len(x.shape) > 1:
        x = np.ravel(x)
        y = np.ravel(y)

    N = len(x)
    l = int((n+1)*(n+2)/2)		# Number of elements in beta
    X = np.ones((N,l))

    for i in range(1,n+1):
        q = int((i)*(i+1)/2)
        for k in range(i+1):
            X[:,q+k] = (x**(i-k))*(y**k)

    return X

In [ ]:
terrain1 = imread("iowa.tif")
terrain2 = imread("rothrock_state_forest_PA.tif")

In [ ]:
terrain1 = terrain1[::10, ::10]
terrain2 = terrain2[::10, ::10]

In [ ]:
fig,ax = plt.subplots(1, 2, figsize=(15,20))

ax[0].set_title("West Central Iowa")
plot1 = ax[0].imshow(terrain1, cmap='coolwarm', vmin=0, vmax=650)
ax[0].set_xlabel("X (Longitude, Seconds)")
ax[0].set_ylabel("Y (Latitude, Seconds)")
scalebar1 = ScaleBar(30.87, location='lower right')
ax[0].add_artist(scalebar1)

divider = make_axes_locatable(ax[0])
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(plot1, cax=cax, orientation='vertical', label='elevation (m)')

ax[1].set_title("Rothrock State Forest, Pennsylvania")
plot2 = ax[1].imshow(terrain2, cmap='coolwarm', vmin=0, vmax=650)
ax[1].set_xlabel("X (Longitude, Seconds)")
ax[1].set_ylabel("Y (Latitude, Seconds)")
scalebar2 = ScaleBar(30.87, location='lower right')
ax[1].add_artist(scalebar2)

divider = make_axes_locatable(ax[1])
cax = divider.append_axes('right', size='5%', pad=0.05)
fig.colorbar(plot2, cax=cax, orientation='vertical', label='elevation (m)')

plt.tight_layout()

In [ ]:
def real_data_OLS(data, npointsx, npointsy, lower_poly, upper_poly, noise, sigma, scaling):

    test_MSEs = []
    test_R2s = []
    train_MSEs = []
    train_R2s = []
    betas = []

    poly_order_range = range(1,6)

    x = np.linspace(0, 1, npointsx)
    y = np.linspace(0, 1, npointsy)

    x, y = np.meshgrid(x,y)

    z = data
    #z += noise*np.random.normal(0, sigma, (len(x),len(x)))
    z = np.ravel(z)
    z = (z-z.mean())/z.std()
    count = 0
    for order in poly_order_range:
        X = design_matrix(x, y, order)

        X_train, X_test, z_train, z_test = train_test_split(X, z, test_size=0.2)
        if scaling == 1:

            X_train = X_train - np.mean(X_train, axis=0)
            X_test = X_test - np.mean(X_test, axis=0)
            z_train = z_train - np.mean(z_train, axis=0)
            z_test = z_test - np.mean(z_test, axis=0)

        beta = np.linalg.pinv(X_train.T @ X_train) @ X_train.T @ z_train
        z_tilde = X_train @ beta
        z_pred = X_test @ beta

        test_MSEs.append(MSE(z_pred, z_test))
        train_MSEs.append(MSE(z_train,z_tilde))
        test_R2s.append(R2(z_test,z_pred))
        train_R2s.append(R2(z_train,z_tilde))
        betas.append(beta)
        #print(MSE(z_pred, z_test))
        count += 1
        print(count)

    return test_MSEs, test_R2s, train_MSEs, train_R2s, betas

In [ ]:
npointsx = terrain1.shape[0]
npointsy = terrain1.shape[1]
lower_poly = 1
upper_poly = 5
noise = 0
sigma = 0.1
scaling = 0

test_MSEs1, test_R2s1, train_MSEs1, train_R2s1, betas1 = real_data_OLS(terrain1, npointsx, npointsy, lower_poly, upper_poly, noise, sigma, scaling)

In [ ]:
npointsx = terrain2.shape[0]
npointsy = terrain2.shape[1]
lower_poly = 1
upper_poly = 5
noise = 0
sigma = 0.1
scaling = 0

test_MSEs2, test_R2s2, train_MSEs2, train_R2s2, betas2 = real_data_OLS(terrain2, npointsx, npointsy, lower_poly, upper_poly, noise, sigma, scaling)

In [ ]:
fig,ax = plt.subplots(2,2, figsize=(12,7))
poly_order_range = range(lower_poly, upper_poly+1)
lw=2
dashl = 4
dashs = 7
ax[0,0].plot(poly_order_range, train_MSEs1, '-o', lw=lw, color='dodgerblue', label='train')
ax[0,0].plot(poly_order_range, test_MSEs1, '--o', lw=lw, dashes=(dashl, dashs), color='darkorange', label='test')
ax[0,0].xaxis.set_major_locator(MaxNLocator(integer=True))
#ax[0,0].set_xlabel('Polynomial Order')
ax[0,0].set_ylabel('MSE')
ax[0,0].set_title('West Central Iowa')
ax[0,0].legend()
ax[0,0].set_ylim(-0.05, 1.05)

ax[0,1].plot(poly_order_range, train_MSEs2, '-o', lw=lw, color='dodgerblue', label='train')
ax[0,1].plot(poly_order_range, test_MSEs2, '--o', lw=lw, dashes=(dashl, dashs), color='darkorange', label='test')
ax[0,1].xaxis.set_major_locator(MaxNLocator(integer=True))
#ax[0,1].set_xlabel('Polynomial Order')
ax[0,1].set_ylabel('MSE')
ax[0,1].set_title('Rothrock State Forest, Pennsylvania')
ax[0,1].legend()
ax[0,1].set_ylim(-0.05, 1.05)

ax[1,0].plot(poly_order_range, train_R2s1, '-o',lw=lw, color='red', label='train')
ax[1,0].plot(poly_order_range, test_R2s1, '--o',lw=lw, dashes=(dashl, dashs), color='green', label='test')
ax[1,0].xaxis.set_major_locator(MaxNLocator(integer=True))
ax[1,0].set_xlabel('Polynomial Order')
ax[1,0].set_ylabel('R2')
#ax[1,0].set_title('Sangre de Cristo Mountains')
ax[1,0].legend()
ax[1,0].set_ylim(-0.05, 1.05)

ax[1,1].plot(poly_order_range, train_R2s2, '-o', lw=lw, color='red', label='train')
ax[1,1].plot(poly_order_range, test_R2s2, '--o', lw=lw, dashes=(dashl, dashs), color='green', label='test')
ax[1,1].xaxis.set_major_locator(MaxNLocator(integer=True))
ax[1,1].set_xlabel('Polynomial Order')
ax[1,1].set_ylabel('R2')
#ax[1,1].set_title('Rothrock State Forest')
ax[1,1].legend()
ax[1,1].set_ylim(-0.05, 1.05)


plt.tight_layout()

# Ridge Regression

In [ ]:
def real_data_ridge(data, npointsx, npointsy, lower, upper, noise, sigma, nlambdas, scaling):

    np.random.seed(1)

    test_MSEs = []
    train_MSEs = []

    poly_order_range = range(lower, upper + 1)

    x = np.linspace(0, 1, npointsx)
    y = np.linspace(0, 1, npointsy)
    x_gr, y_gr = np.meshgrid(x,y)

    z = data
    #z += noise*np.random.normal(0, sigma, (len(x),len(x)))
    #z = np.ravel(z)
    z = (z-z.mean())/z.std()
    count = 0
    for order in poly_order_range:
        X = design_matrix(x, y, order)
        X_train, X_test, z_train, z_test = train_test_split(X, z, test_size = 0.2, random_state = 1)

        if scaling == 1:
            X_train = X_train - np.mean(X_train, axis=0)
            X_test = X_test - np.mean(X_test, axis=0)
            z_train = z_train - np.mean(z_train, axis=0)
            z_test = z_test - np.mean(z_test, axis=0)

        I = np.eye(X_train.shape[1],X_train.shape[1])

        test_MSEs_temp = np.zeros(nlambdas)
        train_MSEs_temp = np.zeros(nlambdas)

        lambdas = np.logspace(-4, 1, nlambdas)

        for i in range(nlambdas):
            lmb = lambdas[i]

            Ridgebeta = np.linalg.pinv(X_train.T @ X_train+lmb*I) @ X_train.T @ z_train

            ztildeRidge = X_train @ Ridgebeta
            zpredictRidge = X_test @ Ridgebeta

            test_MSEs_temp[i] = MSE(z_test,zpredictRidge)
            train_MSEs_temp[i] = MSE(z_train,ztildeRidge)

        train_MSEs.append(train_MSEs_temp)
        test_MSEs.append(test_MSEs_temp)
        count += 1
        print(count)

    train_MSEs = np.array(train_MSEs)
    test_MSEs = np.array(test_MSEs)

    return test_MSEs, train_MSEs

In [ ]:
npointsx = terrain1.shape[0]
npointsy = terrain1.shape[1]
lower_poly = 1
upper_poly = 5
noise = 0
sigma = 0
scaling = 1
nlambdas = 20
lambdas = np.logspace(-4, 1, nlambdas)
test_MSEs, train_MSEs = real_data_ridge(terrain1, npointsx, npointsy, lower_poly, upper_poly, noise, sigma, nlambdas, scaling)

In [ ]:
fig, ax = plt.subplots(figsize=(20,7))
sns.heatmap(train_MSEs, annot=True, ax=ax, cmap="Blues", cbar_kws={'label': 'MSE'})
ax.set_title("Iowa - Training MSE for Ridge Regression")
ax.set_xticklabels(lambdas.round(4))
ax.set_ylabel("Polynomial Degree")
ax.set_xlabel("$\\alpha$")

In [ ]:
fig, ax = plt.subplots(figsize=(20,7))
sns.heatmap(test_MSEs, annot=True, ax=ax, cmap="Blues", cbar_kws={'label': 'MSE'})
ax.set_title("Iowa - Testing MSE for Ridge Regression")
ax.set_xticklabels(lambdas.round(4))
ax.set_ylabel("Polynomial Degree")
ax.set_xlabel("$\\alpha$")

In [ ]:
npointsx = terrain2.shape[0]
npointsy = terrain2.shape[1]
lower_poly = 1
upper_poly = 5
noise = 0
sigma = 0
scaling = 1
nlambdas = 20
lambdas = np.logspace(-4, 1, nlambdas)
test_MSEs, train_MSEs = real_data_ridge(terrain2, npointsx, npointsy, lower_poly, upper_poly, noise, sigma, nlambdas, scaling)

In [ ]:
fig, ax = plt.subplots(figsize=(20,7))
sns.heatmap(train_MSEs, annot=True, ax=ax, cmap="Blues", cbar_kws={'label': 'MSE'})
ax.set_title("Pennsylvania - Training MSE for Ridge Regression")
ax.set_xticklabels(lambdas.round(4))
ax.set_ylabel("Polynomial Degree")
ax.set_xlabel("$\\alpha$")

In [ ]:
fig, ax = plt.subplots(figsize=(20,7))
sns.heatmap(test_MSEs, annot=True, ax=ax, cmap="Blues", cbar_kws={'label': 'MSE'})
ax.set_title("Pennsylvania - Testing MSE for Ridge Regression")
ax.set_xticklabels(lambdas.round(4))
ax.set_ylabel("Polynomial Degree")
ax.set_xlabel("$\\alpha$")

# Lasso Regression

In [ ]:
def real_data_lasso(data, npointsx, npointsy, lower_poly, upper_poly, noise, sigma, nalphas, scaling):

    np.random.seed(1)

    test_MSEs = []
    train_MSEs = []

    poly_order_range = range(lower_poly, upper_poly + 1)

    x = np.linspace(0, 1, npointsx)
    y = np.linspace(0, 1, npointsy)
    x_gr, y_gr = np.meshgrid(x,y)

    z = data
    #z += noise*np.random.normal(0, sigma, (len(x),len(x)))
    #z = np.ravel(z)
    z = (z-z.mean())/z.std()
    count = 0
    for order in poly_order_range:
        X = design_matrix(x, y, order)
        X_train, X_test, z_train, z_test = train_test_split(X, z, test_size = 0.2, random_state = 1)

        if scaling == 1:
            X_train = X_train - np.mean(X_train, axis=0)
            X_test = X_test - np.mean(X_test, axis=0)
            z_train = z_train - np.mean(z_train, axis=0)
            z_test = z_test - np.mean(z_test, axis=0)

        I = np.eye(X_train.shape[1],X_train.shape[1])

        test_MSEs_temp = np.zeros(nalphas)
        train_MSEs_temp = np.zeros(nalphas)
        test_R2s_temp = np.zeros(nalphas)
        train_R2s_temp = np.zeros(nalphas)

        alphas = np.logspace(-4, 1, nalphas)
        #print(alphas)

        for i in range(nalphas):
            alp = alphas[i]
            model = Lasso(alpha = alp)
            model.fit(X_train, z_train)

            ztilde_Lasso = model.predict(X_train)
            zpredict_Lasso = model.predict(X_test)

            test_MSEs_temp[i] = MSE(z_test,zpredict_Lasso)
            train_MSEs_temp[i] = MSE(z_train,ztilde_Lasso)

        train_MSEs.append(train_MSEs_temp)
        test_MSEs.append(test_MSEs_temp)

        count += 1
        print(count)

    train_MSEs = np.array(train_MSEs)
    test_MSEs = np.array(test_MSEs)

    return test_MSEs, train_MSEs

In [ ]:
npointsx = terrain1.shape[0]
npointsy = terrain1.shape[1]
lower_poly = 1
upper_poly = 5
noise = 0
sigma = 0
scaling = 1
nalphas = 20
test_MSEs, train_MSEs = real_data_lasso(terrain1, npointsx, npointsy, lower_poly, upper_poly, noise, sigma, nalphas, scaling)

In [ ]:
fig, ax = plt.subplots(figsize=(20,7))
sns.heatmap(train_MSEs, annot=True, ax=ax, cmap="Blues", cbar_kws={'label': 'MSE'})
ax.set_title("Iowa - Training MSE for Lasso Regression")
alphas = np.logspace(-4, 1, nalphas)
ax.set_xticklabels(alphas.round(4))
ax.set_ylabel("Polynomial Degree")
ax.set_xlabel("$\\alpha$")

In [ ]:
fig, ax = plt.subplots(figsize=(20,7))
sns.heatmap(test_MSEs, annot=True, ax=ax, cmap="Blues", cbar_kws={'label': 'MSE'})
ax.set_title("Iowa - Testing MSE for Lasso Regression")
ax.set_xticklabels(alphas.round(4))
ax.set_ylabel("Polynomial Degree")
ax.set_xlabel("$\\alpha$")

In [ ]:
npointsx = terrain2.shape[0]
npointsy = terrain2.shape[1]
lower_poly = 1
upper_poly = 5
noise = 0
sigma = 0
scaling = 1
nalphas = 20
test_MSEs, train_MSEs = real_data_lasso(terrain2, npointsx, npointsy, lower_poly, upper_poly, noise, sigma, nalphas, scaling)

In [ ]:
fig, ax = plt.subplots(figsize=(20,7))
sns.heatmap(train_MSEs, annot=True, ax=ax, cmap="Blues", cbar_kws={'label': 'MSE'})
ax.set_title("Pennsylvania - Training MSE for Lasso Regression")
ax.set_xticklabels(alphas.round(4))
ax.set_ylabel("Polynomial Degree")
ax.set_xlabel("$\\alpha$")

In [ ]:
fig, ax = plt.subplots(figsize=(20,7))
sns.heatmap(test_MSEs, annot=True, ax=ax, cmap="Blues", cbar_kws={'label': 'MSE'})
ax.set_title("Pennsylvania - Testing MSE for Lasso Regression")
ax.set_xticklabels(alphas.round(4))
ax.set_ylabel("Polynomial Degree")
ax.set_xlabel("$\\alpha$")